In [1]:
#!pip install wrapt

In [2]:
import os
import logging
import sys
import contextlib
from pprint import pprint
import wrapt  
import functools

# Задача 1 Применение метаклассов. 

Три варианта решения. Наиболее оптимальным видится третий вариант.

Вариант 1. В `__new__()` метакласса AttrLoggingMeta:
* методы и аттрибуты начинающиеся с '__' пропускаются. Предполагаем, что приватные методы не требуется логгировать.
* остальные методы оборачиваются декоратором log_access() - статическим методом класса AttrLoggingMeta
* для аттрибутов:
    * создается аттрибут класса с именем по масске `f'__logged_by_ALG_{исходное имя аттрибута}'`
    * сам же атрибут подменяется на экземпляр класса property(), для которого геттер и сеттер созданы с логгированием

Особенности и ограничения:
* аттрибуты экземпляра класса (добавляемые в `__init__()`) логгировать не будут, т.к. появляются после того, как отработает `__new__()` метакласса

Вриант 2. Отличия от варианта 1: логгирование чтения и обновления аттрибутов делается через декорирование методов `__getattribute__` и `__setattr__`, 
которые должны быть определены в классе.

Вариант 3. Отличия от варианта 2: Избавился от обязательного объявления методов `__getattribute__` и `__setattr__` в логгируемом классе


### Вариант 1

In [3]:
class AttrLoggingMeta(type):
    """Мета-класс с контролем доступа к методам и чтения-записи атрибутов"""
    def __new__(mcs, name, bases, attrs, **extra_kwargs):

        need_change_attr = []

        for attr, method in attrs.items():  
            if not attr.startswith('__'): # пропустим все приватные аттрибуты и методы
                if callable(method):
                    # оборачиваем все методы декоратором-логгером
                    attrs[attr] = AttrLoggingMeta.log_access(method) 
                else:
                    # запомним аттрибуты
                    need_change_attr.append(attr)
                
        # для каждого аттрибута создадим property со специальными сеттером и геттором
        for attr in need_change_attr:
            attrs[f'__logged_by_ALG_{attr}'] = attrs[attr]
            attrs[attr] = property(fget=AttrLoggingMeta.__create_getter(attr), fset=AttrLoggingMeta.__create_setter(attr))

        cls_obj = super().__new__(mcs, name, bases, attrs)  
        return cls_obj

    def __init__(cls, name, bases, attrs, **extra_kwargs):  
        super().__init__(cls)  

    @classmethod  
    def __prepare__(mcs, cls, bases, **extra_kwargs):  
        return super().__prepare__(mcs, cls, bases, **extra_kwargs)  
        
    @staticmethod
    def log_access(func):
        """Логгер доступа к методу"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f"AttrLoggingMeta: Calling method {func.__qualname__}()")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def __create_getter(name):
        """Создание геттера-логгера"""
        body_getter =f"""
def __getter(self):
    print(f"AttrLoggingMeta: Reading attribute: {name}")
    return self.__logged_by_ALG_{name}
        """
        #print(body_getter)
        exec(body_getter)
        return locals()[f'__getter']

    @staticmethod
    def __create_setter(name):
        """Создание сеттера-логгера"""
        body_setter =f"""
def __setter(self, value):
    print(f"AttrLoggingMeta: Writing attribute: {name} with value {{value}}")
    self.__logged_by_ALG_{name} = value
        """
        #print(body_setter)
        exec(body_setter)
        return locals()[f'__setter']

    def __call__(cls, *args, **kwargs):  
        return super().__call__(*args, **kwargs)    


In [4]:
class LoggedClass(metaclass = AttrLoggingMeta): 
    """Тестовый класс"""
    class_attr = "class_attr_value"
    
    def __new__(cls, attr1, attr2, attr3):  
        return super().__new__(cls)  
        
    def __init__(self, attr1, attr2, attr3):
        self.instance_attr1 = attr1
        self.__attr2 = attr2
        self.__attr3 = attr3

    def my_method(self):
        return('LoggedClass.my_method() is runned()')

    def set_instance_attr1(self, value):
        self.instance_attr1 = value
        print('LoggedClass.set_instance_attr1() is runned()')
    
    # read-only property класса
    @property
    def class_attr2(self):
        print('LoggedClass.attr2 is read()')
        return self.__attr2
    
    # mutable property класса
    @property 
    def class_attr3(self):
        print('LoggedClass.attr3 is read()')
        # значение оборачивается в звездочки, чтобы убедиться, что заданный в классе геттер работает
        return f'**{self.__attr3}**'
    
    @class_attr3.setter
    def class_attr3(self, value):
        print('LoggedClass.attr3 is write()')
        # значение свойства "удваивается", чтобы проверить, что заданный в классе сеттер работает
        self.__attr3 = f'{value}_{value}'
    
lc = LoggedClass(attr1="attr1_value", attr2="attr2_value", attr3 = "attr3_value")    

#### Проверить логгирование обращений к методам

In [5]:
# Проверяем логгирование возвращающего метода
print(lc.my_method())


AttrLoggingMeta: Calling method LoggedClass.my_method()
LoggedClass.my_method() is runned()


In [6]:
# Проверяем логгирование метода без возвращаемого значения, но с параметором
print("lc.instance_attr1 = " + lc.instance_attr1)
lc.set_instance_attr1("new_instance_attr1_value")
print("lc.instance_attr1 = " + lc.instance_attr1)

lc.instance_attr1 = attr1_value
AttrLoggingMeta: Calling method LoggedClass.set_instance_attr1()
LoggedClass.set_instance_attr1() is runned()
lc.instance_attr1 = new_instance_attr1_value


#### Проверить логгирование обращений к class_attr

In [7]:
# чтение
print(f'lc.class_attr={lc.class_attr}')

AttrLoggingMeta: Reading attribute: class_attr
lc.class_attr=class_attr_value


In [8]:
# изменение
lc.class_attr = "class_attr_new"


AttrLoggingMeta: Writing attribute: class_attr with value class_attr_new


In [9]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.class_attr={lc.class_attr}')


AttrLoggingMeta: Reading attribute: class_attr
lc.class_attr=class_attr_new


#### Проверить логгирование обращений к instance_attr1

Логов быть не должно, т.к. это не аттрибут класса, а атрибут экземпляра

In [10]:
# чтение
print(f'lc.instance_attr1={lc.instance_attr1}')

lc.instance_attr1=new_instance_attr1_value


In [11]:
# изменение
lc.instance_attr1 = "instance_attr1_new"

In [12]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.instance_attr1={lc.instance_attr1}\n')

lc.instance_attr1=instance_attr1_new



#### Проверить логгирование обращений к class_attr2

In [13]:
# чтение
print(f'lc.class_attr2={lc.class_attr2}')

AttrLoggingMeta: Reading attribute: class_attr2
LoggedClass.attr2 is read()
lc.class_attr2=attr2_value


In [14]:
# Изменение. Д.б. ошибка, т.к. у свойста не задан сеттер
try:
    lc.class_attr2 = "class_attr2_new"
except Exception as e:
   print(f'Ошибка: {e}')

AttrLoggingMeta: Writing attribute: class_attr2 with value class_attr2_new
Ошибка: property '__logged_by_ALG_class_attr2' of 'LoggedClass' object has no setter


#### Проверить логгирование обращений к class_attr3

In [15]:
# чтение
print(f'lc.class_attr3={lc.class_attr3}')

AttrLoggingMeta: Reading attribute: class_attr3
LoggedClass.attr3 is read()
lc.class_attr3=**attr3_value**


In [16]:
# изменение
lc.class_attr3 = "class_attr3_new"

AttrLoggingMeta: Writing attribute: class_attr3 with value class_attr3_new
LoggedClass.attr3 is write()


In [17]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.class_attr3={lc.class_attr3}')

AttrLoggingMeta: Reading attribute: class_attr3
LoggedClass.attr3 is read()
lc.class_attr3=**class_attr3_new_class_attr3_new**


In [18]:
LoggedClass.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Тестовый класс',
              'class_attr': <property at 0x71c8a9b17380>,
              '__new__': <staticmethod(<function LoggedClass.__new__ at 0x71c8a9b25b20>)>,
              '__init__': <function __main__.LoggedClass.__init__(self, attr1, attr2, attr3)>,
              'my_method': <function __main__.LoggedClass.my_method(self)>,
              'set_instance_attr1': <function __main__.LoggedClass.set_instance_attr1(self, value)>,
              'class_attr2': <property at 0x71c8a9b17470>,
              'class_attr3': <property at 0x71c8a9b17560>,
              '__logged_by_ALG_class_attr': 'class_attr_value',
              '__logged_by_ALG_class_attr2': <property at 0x71c8aa599300>,
              '__logged_by_ALG_class_attr3': <property at 0x71c8a9b16c50>,
              '__dict__': <attribute '__dict__' of 'LoggedClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'LoggedClass' objects>})

In [19]:
lc.__dict__

{'instance_attr1': 'instance_attr1_new',
 '_LoggedClass__attr2': 'attr2_value',
 '_LoggedClass__attr3': 'class_attr3_new_class_attr3_new',
 '__logged_by_ALG_class_attr': 'class_attr_new'}

### Вариант 2

In [20]:
class AttrLoggingMeta2(type):
    """Мета-класс с контролем доступа к методам и чтения-записи атрибутов"""
    def __new__(mcs, name, bases, attrs, **extra_kwargs):

        for attr, method in attrs.items():  
            if attr == '__getattribute__':
                attrs[attr] = AttrLoggingMeta2.log_read(method) 
            elif attr == '__setattr__':
                attrs[attr] = AttrLoggingMeta2.log_write(method) 
            elif not attr.startswith('__'): # пропустим все приватные аттрибуты и методы
                if callable(method):
                    # оборачиваем все методы декоратором-логгером
                    attrs[attr] = AttrLoggingMeta2.log_access(method) 

        cls_obj = super().__new__(mcs, name, bases, attrs)  

        return cls_obj

    def __init__(cls, name, bases, attrs, **extra_kwargs):  
        super().__init__(cls)  

    @classmethod  
    def __prepare__(mcs, cls, bases, **extra_kwargs):  
        return super().__prepare__(mcs, cls, bases, **extra_kwargs)  
        
    @staticmethod
    def log_access(func):
        """Логгер доступа к методу"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f"AttrLoggingMeta2: Calling method {func.__qualname__}()")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def log_read(func):
        """Логгер чтения атрибутов"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if not args[1].startswith('__'): # пропустим все приватные аттрибуты
                print(f"AttrLoggingMeta2: read attribute {args[1]}")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def log_write(func):
        """Логгер записи атрибутов"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if not args[1].startswith('__'): # пропустим все приватные аттрибуты
                print(f"AttrLoggingMeta2: update attribute {args[1]} to {args[2]}")
            return func(*args, **kwargs)
        return wrapper

    def __call__(cls, *args, **kwargs):  
        return super().__call__(*args, **kwargs)    

In [21]:

class LoggedClass2(metaclass = AttrLoggingMeta2): 
    """Тестовый класс"""
    class_attr = "class_attr_value"
    
    def __new__(cls, attr1, attr2, attr3):  
        return super().__new__(cls)  
        
    def __init__(self, attr1, attr2, attr3):
        self.instance_attr1 = attr1
        self.__attr2 = attr2
        self.__attr3 = attr3

    def my_method(self):
        return('LoggedClass.my_method() is runned()')

    def set_instance_attr1(self, value):
        self.instance_attr1 = value
        print('LoggedClass.set_instance_attr1() is runned()')
    
    # read-only property класса
    @property
    def class_attr2(self):
        print('LoggedClass.attr2 is read()')
        return self.__attr2
    
    # mutable property класса
    @property 
    def class_attr3(self):
        print('LoggedClass.attr3 is read()')
        # значение оборачивается в звездочки, чтобы убедиться, что заданный в классе геттер работает
        return f'**{self.__attr3}**'
    
    @class_attr3.setter
    def class_attr3(self, value):
        print('LoggedClass.attr3 is write()')
        # значение свойства "удваивается", чтобы проверить, что заданный в классе сеттер работает
        self.__attr3 = f'{value}_{value}'
        
    def __getattribute__(self, name):
        return super().__getattribute__(name)

    def __setattr__(self, name, value):
        return super().__setattr__(name, value)

lc2 = LoggedClass2(attr1="attr1_value", attr2="attr2_value", attr3 = "attr3_value")        

AttrLoggingMeta2: update attribute instance_attr1 to attr1_value
AttrLoggingMeta2: update attribute _LoggedClass2__attr2 to attr2_value
AttrLoggingMeta2: update attribute _LoggedClass2__attr3 to attr3_value


#### Проверить логгирование обращений к методам

In [22]:
# Проверяем логгирование возвращающего метода
print(lc2.my_method())


AttrLoggingMeta2: read attribute my_method
AttrLoggingMeta2: Calling method LoggedClass2.my_method()
LoggedClass.my_method() is runned()


In [23]:
# Проверяем логгирование метода без возвращаемого значения, но с параметором
print("lc2.instance_attr1 = " + lc2.instance_attr1)
lc2.set_instance_attr1("new_instance_attr1_value")
print("lc2.instance_attr1 = " + lc2.instance_attr1)

AttrLoggingMeta2: read attribute instance_attr1
lc2.instance_attr1 = attr1_value
AttrLoggingMeta2: read attribute set_instance_attr1
AttrLoggingMeta2: Calling method LoggedClass2.set_instance_attr1()
AttrLoggingMeta2: update attribute instance_attr1 to new_instance_attr1_value
LoggedClass.set_instance_attr1() is runned()
AttrLoggingMeta2: read attribute instance_attr1
lc2.instance_attr1 = new_instance_attr1_value


#### Проверить логгирование обращений к class_attr

In [24]:
# чтение
print(f'lc2.class_attr={lc2.class_attr}')

AttrLoggingMeta2: read attribute class_attr
lc2.class_attr=class_attr_value


In [25]:
# изменение
lc2.class_attr = "class_attr_new"


AttrLoggingMeta2: update attribute class_attr to class_attr_new


In [26]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc2.class_attr={lc2.class_attr}')


AttrLoggingMeta2: read attribute class_attr
lc2.class_attr=class_attr_new


#### Проверить логгирование обращений к instance_attr1

Логов быть не должно, т.к. это не аттрибут класса, а атрибут экземпляра

In [27]:
# чтение
print(f'lc2.instance_attr1={lc2.instance_attr1}')

AttrLoggingMeta2: read attribute instance_attr1
lc2.instance_attr1=new_instance_attr1_value


In [28]:
# изменение
lc2.instance_attr1 = "instance_attr1_new"

AttrLoggingMeta2: update attribute instance_attr1 to instance_attr1_new


In [29]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc2.instance_attr1={lc2.instance_attr1}\n')

AttrLoggingMeta2: read attribute instance_attr1
lc2.instance_attr1=instance_attr1_new



#### Проверить логгирование обращений к class_attr2

In [30]:
# чтение
print(f'lc2.class_attr2={lc2.class_attr2}')

AttrLoggingMeta2: read attribute class_attr2
LoggedClass.attr2 is read()
AttrLoggingMeta2: read attribute _LoggedClass2__attr2
lc2.class_attr2=attr2_value


In [31]:
# Изменение. Д.б. ошибка, т.к. у свойста не задан сеттер
try:
    lc2.class_attr2 = "class_attr2_new"
except Exception as e:
   print(f'Ошибка: {e}')

AttrLoggingMeta2: update attribute class_attr2 to class_attr2_new
Ошибка: property 'class_attr2' of 'LoggedClass2' object has no setter


#### Проверить логгирование обращений к class_attr3

In [32]:
# чтение
print(f'lc2.class_attr3={lc2.class_attr3}')

AttrLoggingMeta2: read attribute class_attr3
LoggedClass.attr3 is read()
AttrLoggingMeta2: read attribute _LoggedClass2__attr3
lc2.class_attr3=**attr3_value**


In [33]:
# изменение
lc2.class_attr3 = "class_attr3_new"

AttrLoggingMeta2: update attribute class_attr3 to class_attr3_new
LoggedClass.attr3 is write()
AttrLoggingMeta2: update attribute _LoggedClass2__attr3 to class_attr3_new_class_attr3_new


In [34]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc2.class_attr3={lc2.class_attr3}')

AttrLoggingMeta2: read attribute class_attr3
LoggedClass.attr3 is read()
AttrLoggingMeta2: read attribute _LoggedClass2__attr3
lc2.class_attr3=**class_attr3_new_class_attr3_new**


In [35]:
LoggedClass2.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Тестовый класс',
              'class_attr': 'class_attr_value',
              '__new__': <staticmethod(<function LoggedClass2.__new__ at 0x71c8a9b26ac0>)>,
              '__init__': <function __main__.LoggedClass2.__init__(self, attr1, attr2, attr3)>,
              'my_method': <function __main__.LoggedClass2.my_method(self)>,
              'set_instance_attr1': <function __main__.LoggedClass2.set_instance_attr1(self, value)>,
              'class_attr2': <property at 0x71c8a9b371a0>,
              'class_attr3': <property at 0x71c8a9b35a30>,
              '__getattribute__': <function __main__.LoggedClass2.__getattribute__(self, name)>,
              '__setattr__': <function __main__.LoggedClass2.__setattr__(self, name, value)>,
              '__dict__': <attribute '__dict__' of 'LoggedClass2' objects>,
              '__weakref__': <attribute '__weakref__' of 'LoggedClass2' objects>})

In [36]:
lc2.__dict__

{'instance_attr1': 'instance_attr1_new',
 '_LoggedClass2__attr2': 'attr2_value',
 '_LoggedClass2__attr3': 'class_attr3_new_class_attr3_new',
 'class_attr': 'class_attr_new'}

### Вариант 3

In [37]:

class AttrLoggingMeta3(type):
    """Мета-класс с контролем доступа к методам и чтения-записи атрибутов"""
    def __new__(mcs, name, bases, attrs, **extra_kwargs):

        # Если нето __getattribute__ и __setattr__ - то создать их
        if '__getattribute__' not in attrs.keys():
            attrs['__getattribute__'] = mcs.__getattr2__
        if '__setattr__' not in attrs.keys():
            attrs['__setattr__'] = mcs.__setattr2__

        for attr, method in attrs.items():  
            if attr == '__getattribute__':
                attrs[attr] = AttrLoggingMeta3.log_read(method) 
            elif attr == '__setattr__':
                attrs[attr] = AttrLoggingMeta3.log_write(method) 
            elif not attr.startswith('__'): # пропустим все приватные аттрибуты и методы
                if callable(method):
                    # оборачиваем все методы декоратором-логгером
                    attrs[attr] = AttrLoggingMeta3.log_access(method) 

        cls_obj = super().__new__(mcs, name, bases, attrs)  

        return cls_obj

    @staticmethod
    def __setattr2__(self, key, value):
        object.__setattr__(self, key, value)

    @staticmethod
    def __getattr2__(self, item):
        return object.__getattribute__(self, item)
        
    @staticmethod
    def log_access(func):
        """Логгер доступа к методу"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f"AttrLoggingMeta3: Calling method {func.__qualname__}()")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def log_read(func):
        """Логгер чтения атрибутов"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if not args[1].startswith('__'): # пропустим все приватные аттрибуты
                print(f"AttrLoggingMeta3: read attribute {args[1]}")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def log_write(func):
        """Логгер записи атрибутов"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if not args[1].startswith('__'): # пропустим все приватные аттрибуты
                print(f"AttrLoggingMeta3: update attribute {args[1]} to {args[2]}")
            return func(*args, **kwargs)
        return wrapper


AttrLoggingMeta3.__dict__    

mappingproxy({'__module__': '__main__',
              '__doc__': 'Мета-класс с контролем доступа к методам и чтения-записи атрибутов',
              '__new__': <staticmethod(<function AttrLoggingMeta3.__new__ at 0x71c8a9b640e0>)>,
              '__setattr2__': <staticmethod(<function AttrLoggingMeta3.__setattr2__ at 0x71c8a9b64180>)>,
              '__getattr2__': <staticmethod(<function AttrLoggingMeta3.__getattr2__ at 0x71c8a9b64220>)>,
              'log_access': <staticmethod(<function AttrLoggingMeta3.log_access at 0x71c8a9b642c0>)>,
              'log_read': <staticmethod(<function AttrLoggingMeta3.log_read at 0x71c8a9b64360>)>,
              'log_write': <staticmethod(<function AttrLoggingMeta3.log_write at 0x71c8a9b64400>)>})

In [38]:

class LoggedClass3(metaclass = AttrLoggingMeta3):
    """Тестовый класс"""
    class_attr = "class_attr_value"
        
    def __init__(self, attr1, attr2, attr3):
        self.instance_attr1 = attr1
        self.__attr2 = attr2
        self.__attr3 = attr3

    def my_method(self):
        return('LoggedClass.my_method() is runned()')

    def set_instance_attr1(self, value):
        self.instance_attr1 = value
        print('LoggedClass.set_instance_attr1() is runned()')
    
    # read-only property класса
    @property
    def class_attr2(self):
        print('LoggedClass.attr2 is read()')
        return self.__attr2
    
    # mutable property класса
    @property 
    def class_attr3(self):
        print('LoggedClass.attr3 is read()')
        # значение оборачивается в звездочки, чтобы убедиться, что заданный в классе геттер работает
        return f'**{self.__attr3}**'
    
    @class_attr3.setter
    def class_attr3(self, value):
        print('LoggedClass.attr3 is write()')
        # значение свойства "удваивается", чтобы проверить, что заданный в классе сеттер работает
        self.__attr3 = f'{value}_{value}'


In [39]:
LoggedClass3.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Тестовый класс',
              'class_attr': 'class_attr_value',
              '__init__': <function __main__.LoggedClass3.__init__(self, attr1, attr2, attr3)>,
              'my_method': <function __main__.LoggedClass3.my_method(self)>,
              'set_instance_attr1': <function __main__.LoggedClass3.set_instance_attr1(self, value)>,
              'class_attr2': <property at 0x71c8a9b34cc0>,
              'class_attr3': <property at 0x71c8a9b5e7f0>,
              '__getattribute__': <function __main__.AttrLoggingMeta3.__getattr2__(self, item)>,
              '__setattr__': <function __main__.AttrLoggingMeta3.__setattr2__(self, key, value)>,
              '__dict__': <attribute '__dict__' of 'LoggedClass3' objects>,
              '__weakref__': <attribute '__weakref__' of 'LoggedClass3' objects>})

In [40]:
lc3 = LoggedClass3(attr1="attr1_value", attr2="attr2_value", attr3 = "attr3_value")    

AttrLoggingMeta3: update attribute instance_attr1 to attr1_value
AttrLoggingMeta3: update attribute _LoggedClass3__attr2 to attr2_value
AttrLoggingMeta3: update attribute _LoggedClass3__attr3 to attr3_value


#### Проверить логгирование обращений к методам

In [41]:
# Проверяем логгирование возвращающего метода
print(lc3.my_method())

AttrLoggingMeta3: read attribute my_method
AttrLoggingMeta3: Calling method LoggedClass3.my_method()
LoggedClass.my_method() is runned()


In [42]:
# Проверяем логгирование метода без возвращаемого значения, но с параметором
print("lc3.instance_attr1 = " + lc3.instance_attr1)
lc3.set_instance_attr1("new_instance_attr1_value")
print("lc3.instance_attr1 = " + lc3.instance_attr1)

AttrLoggingMeta3: read attribute instance_attr1
lc3.instance_attr1 = attr1_value
AttrLoggingMeta3: read attribute set_instance_attr1
AttrLoggingMeta3: Calling method LoggedClass3.set_instance_attr1()
AttrLoggingMeta3: update attribute instance_attr1 to new_instance_attr1_value
LoggedClass.set_instance_attr1() is runned()
AttrLoggingMeta3: read attribute instance_attr1
lc3.instance_attr1 = new_instance_attr1_value


#### Проверить логгирование обращений к class_attr

In [43]:
# чтение
print(f'lc3.class_attr={lc3.class_attr}')

AttrLoggingMeta3: read attribute class_attr
lc3.class_attr=class_attr_value


In [44]:
# изменение
lc3.class_attr = "class_attr_new"

AttrLoggingMeta3: update attribute class_attr to class_attr_new


In [45]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc3.class_attr={lc3.class_attr}')

AttrLoggingMeta3: read attribute class_attr
lc3.class_attr=class_attr_new


#### Проверить логгирование обращений к instance_attr1

Логов быть не должно, т.к. это не аттрибут класса, а атрибут экземпляра

In [46]:
# чтение
print(f'lc3.instance_attr1={lc3.instance_attr1}')

AttrLoggingMeta3: read attribute instance_attr1
lc3.instance_attr1=new_instance_attr1_value


In [47]:
# изменение
lc3.instance_attr1 = "instance_attr1_new"

AttrLoggingMeta3: update attribute instance_attr1 to instance_attr1_new


In [48]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc3.instance_attr1={lc3.instance_attr1}\n')

AttrLoggingMeta3: read attribute instance_attr1
lc3.instance_attr1=instance_attr1_new



#### Проверить логгирование обращений к class_attr2

In [49]:
# чтение
print(f'lc3.class_attr2={lc3.class_attr2}')

AttrLoggingMeta3: read attribute class_attr2
LoggedClass.attr2 is read()
AttrLoggingMeta3: read attribute _LoggedClass3__attr2
lc3.class_attr2=attr2_value


In [50]:
# Изменение. Д.б. ошибка, т.к. у свойста не задан сеттер
try:
    lc3.class_attr2 = "class_attr2_new"
except Exception as e:
   print(f'Ошибка: {e}')

AttrLoggingMeta3: update attribute class_attr2 to class_attr2_new
Ошибка: property 'class_attr2' of 'LoggedClass3' object has no setter


#### Проверить логгирование обращений к class_attr3

In [51]:
# чтение
print(f'lc3.class_attr3={lc3.class_attr3}')

AttrLoggingMeta3: read attribute class_attr3
LoggedClass.attr3 is read()
AttrLoggingMeta3: read attribute _LoggedClass3__attr3
lc3.class_attr3=**attr3_value**


In [52]:
# изменение
lc3.class_attr3 = "class_attr3_new"

AttrLoggingMeta3: update attribute class_attr3 to class_attr3_new
LoggedClass.attr3 is write()
AttrLoggingMeta3: update attribute _LoggedClass3__attr3 to class_attr3_new_class_attr3_new


In [53]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc3.class_attr3={lc3.class_attr3}')

AttrLoggingMeta3: read attribute class_attr3
LoggedClass.attr3 is read()
AttrLoggingMeta3: read attribute _LoggedClass3__attr3
lc3.class_attr3=**class_attr3_new_class_attr3_new**


# Задача 2 Динамическое создание класса


In [54]:

def  create_class_with_methods(name, attributes, methods):
    # объеденить методы и атрибуты в однин словарь
    new_dict = attributes | methods
    # создать новый класс
    new_class = type(name, (), new_dict)
    return new_class

attributes = { 'species': 'Human', 'age': 25 }
methods = { 'greet': lambda self: f"Hello, I am a {self.species} and I am {self.age} years old." }
DynamicClass = create_class_with_methods('DynamicClass', attributes, methods) 

instance = DynamicClass()
print(instance.greet())

Hello, I am a Human and I am 25 years old.


# Задача 3 Генерация кода


In [55]:
def generate_complex_function(function_name, parameters, function_body):
    """Создать функцию по описанию и вернуть её"""
    # собрать строку с параметрами
    parameters_str = ','.join(parameters)
    # добавить в строки с телом функции отступы
    intendent_function_body = '\n'.join(['    '+line for line in function_body.split('\n')])
    # собрать строку для формирования функции
    exec_str = f"def {function_name}({parameters_str}):\n{intendent_function_body}"
    # создать функцию
    exec(exec_str)
    # извлечь функцию из локальных переменных и вернуть её
    return locals()[function_name]

In [56]:
function_name = 'complex_function'
parameters = ['x', 'y']
function_body = """
if x > y:
   return x - y
else:
   return y - x
"""
complex_func = generate_complex_function(function_name, parameters, function_body)

print(complex_func(10, 5))
print(complex_func(5, 10))

5
5


# 